In [2]:
import numpy as np

In [3]:
def sig_mtx(array, precisao):
    for i in range(len(array)):
        for j in range(len(array[0])):
            array[i][j] = float(np.format_float_positional(array[i][j], unique=False, precision=precisao, fractional=False))
    return array

In [4]:
def sig_row(row, precisao):
    for i in range(len(row)):
        row[i] = float(np.format_float_positional(row[i], unique=False, precision=precisao, fractional=False))
    return row

In [5]:
def sig_num(numero, precisao):
    numero = float(np.format_float_positional(numero, unique=False, precision=precisao, fractional=False))
    return numero

In [65]:
def gauss_com_pivot(matrix, precisao):
    linhas = len(matrix)
    
    if(len(matrix) != len(matrix[0])):
        colunas = len(matrix[0]) - 1
    else:
        colunas = len(matrix)
    
    for i in range (colunas):
        linha_pivot = i + np.argmax(np.abs(matrix[i:, i]))
        print(f'iteração {i}:\n Matrix antes do passo:\n{matrix}')
        
        if (linha_pivot != i):
            matrix[[i,linha_pivot],:] = matrix[[linha_pivot,i],:]
        
        for j in range (i+1, linhas):
            matrix[j] = matrix[j] - sig_row(matrix[j][i]/matrix[i][i]*matrix[i], precisao)
        #print(f'Matrix antes do arredondamento:\n{matrix}')
        matrix = sig_mtx(matrix, precisao)
        print(f'Matrix apos arredondamento:\n{matrix}')
    return matrix

In [68]:
def subs_retroativa(matrix, precisao):
    matrix = gauss_com_pivot(matrix, precisao)
    colunas = len(matrix[0]) - 1
    linhas = len(matrix)
    saida = np.zeros(linhas, dtype='f')
    soma_acumulada = 0
    
    for i in range (linhas -1, -1, -1):
        soma_acumulada = 0
        if (i == colunas - 1):
            saida[i] = sig_num(matrix[linhas - 1][colunas]/matrix[linhas - 1][colunas - 1], 2)
            
        
        #pode fazer isso?
        else:
            for j in range(colunas - 1, -1, -1):
                soma_acumulada= soma_acumulada + matrix[i][j]*saida[j]
                
            saida[i] = (matrix[i][colunas] - soma_acumulada)/matrix[i][i]
        
        print(f'saída {i+1} antes do arredondamento: {saida[i]}')
        saida[i] = sig_num(saida[i], precisao)
        print(f'saída {i+1} depois do arredondamento: {saida[i]}\n')
        
    print(f'saída final:\n{saida}')
    
    #return saida

In [8]:
def crit_sassenfeld(matrix):    
    linhas = len(matrix)
    colunas = len(matrix[0]) - 1
    b = np.zeros(linhas, dtype='f')
    saida = 0
    
    for i in range(linhas):
        temp = 0
        if(i == 0):
            for j in range(1,colunas):
                temp = temp + abs(matrix[0][j])
            b[0] = temp/abs(matrix[0][0])
        else:
            for j in range(i+1, colunas):
                temp = temp + abs(matrix[i][j])
            for k in range(i):
                temp = temp + abs(matrix[i][k]*b[k])
            b[i] = temp/abs(matrix[i][i])
        print(f'b[{i}] = {b[i]}')
    
    saida = np.max(b)
    return saida

In [9]:
def crit_linhas(matrix):
    linhas = len(matrix)
    colunas = len(matrix[0]) - 1    
    saida = True
    temp = 0
    soma_linhas = np.zeros(linhas, dtype='f')
    
    for i in range(linhas):
        temp = 0
        for j in range(colunas):
            if(i != j):
                temp = temp + abs(matrix[i][j])
                
        soma_linhas[i] = temp
        print(f'somatório {i}: {soma_linhas[i]}, diagonal: {abs(matrix[i][i])}')
        
        if(soma_linhas[i] > abs(matrix[i][i])):
            saida = False
    
    return saida

In [105]:
def gauss_seidel(matrix, x_0, iteracoes):
    linhas = len(matrix)
    colunas = len(matrix[0]) - 1    
    x_k = x_0
    x_k1 = np.zeros(linhas, dtype='f')
    b = matrix[:, -1]
    temp = 0
    vi_k1=np.zeros(linhas, dtype='f')
    
    for k in range(iteracoes):
        if(k != 0):
            x_k = np.copy(x_k1)
            x_k1 = np.zeros(linhas, dtype='f')
            
        for i in range(linhas):
            temp = 0
        
            for j in range(colunas):
                if(j < i):
                    temp = temp + matrix[i][j]*x_k1[j]
                    #temp = temp + sig_num(matrix[i][j]*x_k1[j], precisao)
            
                if(j > i):
                    temp = temp + matrix[i][j]*x_k[j]
                    #temp = sig_num(temp + matrix[i][j]*x_k[j], precisao)
            
                
            x_k1[i] = (b[i] - temp)/matrix[i][i]
            #x_k1[i] = sig_num(sig_num((b[i] - temp), 3)/matrix[i][i], precisao)
            
            if(x_k1[i] != 0):
                vi_k1[i] = abs(x_k1[i] - x_k[i])/abs(x_k1[i])
            elif(x_k1[i] == 0 and x_k[i] == 0):
                vi_k1[i] = 0
            elif(x_k1[i] == 0 and x_k[i] != 0):
                vi_k1[i] = 1
        
        print(f'iteração {k+1}')
        print(f'X_{k}: {x_k}\nX_{k+1}: {x_k1}\nVi_{k+1}: {vi_k1}\nVar_{k+1}: {np.max(vi_k1)}')
        
    return x_k1

Questão 1

In [37]:
A = np.array([[-1, 0, 2, 0], [4, 1, 3, 1], [2, 2, -1, 1], [0, 0, 1, 0]], dtype='f')

1.1

In [35]:
print(gauss_com_pivot(A, 10))

[[ 4.          1.          3.          1.        ]
 [ 0.          1.5        -2.5         0.5       ]
 [ 0.          0.          3.1666667   0.16666666]
 [ 0.          0.          0.         -0.05263157]]


In [39]:
1.2

1.2

In [38]:
np.linalg.det(A)

-1.0

1.3

In [44]:
np.linalg.inv(A)

array([[-1.0000000e+00,  0.0000000e+00,  0.0000000e+00,  2.0000000e+00],
       [-2.0000000e+00, -1.0000000e+00,  1.0000000e+00,  8.0000000e+00],
       [-3.5059673e-17, -1.7529837e-17,  8.7649183e-18,  1.0000000e+00],
       [ 6.0000000e+00,  2.0000000e+00, -1.0000000e+00, -1.9000000e+01]],
      dtype=float32)

2

In [47]:
B = np.array([[5, 2, -2, 1], [-2, 5, 2, 0], [1, 2, 5, 0]],dtype='f')

2.1

In [48]:
print(crit_sassenfeld(B))

b[0] = 0.800000011920929
b[1] = 0.7200000286102295
b[2] = 0.4480000138282776
0.8


In [50]:
print(crit_linhas(B))

somatório 0: 4.0, diagonal: 5.0
somatório 1: 4.0, diagonal: 5.0
somatório 2: 3.0, diagonal: 5.0
True


Método de Gauss-Seidel converge

In [52]:
B_2 = np.array([[5, 1, 2, 0], [-2, 5, 2, 1], [-1, 3, 3, 0]],dtype='f')

In [53]:
print(crit_sassenfeld(B_2))

print(crit_linhas(B_2))

b[0] = 0.6000000238418579
b[1] = 0.6399999856948853
b[2] = 0.8399999737739563
0.84
somatório 0: 3.0, diagonal: 5.0
somatório 1: 4.0, diagonal: 5.0
somatório 2: 4.0, diagonal: 3.0
False


3

In [34]:
C = np.array([[2, 1, 6, 3], [4, -2, 1, 2], [1, -5, -2, -4]],dtype='f')

3.1

In [35]:
subs_retroativa(C, 2)

2: 0.0
2: 0.0
2: 0.0
saída 3 antes do arredondamento: 0.0
saída 3 depois do arredondamento: 0.0

1: 0.0
1: 0.0
1: 0.0
saída 2 antes do arredondamento: 1.0
saída 2 depois do arredondamento: 1.0

0: 0.0
0: -2.0
0: -2.0
saída 1 antes do arredondamento: 1.0
saída 1 depois do arredondamento: 1.0

saída final:
[1. 1. 0.]


3.2

In [63]:
print(crit_sassenfeld(C))

b[0] = 3.5
b[1] = 7.5
b[2] = 20.5
20.5


Não satisfaz

In [64]:
C[[0,1], :]  = C[[1, 0],:]
C[[1,2],:] = C[[2,1],:]

In [65]:
print(crit_linhas(C))

somatório 0: 3.0, diagonal: 4.0
somatório 1: 3.0, diagonal: 5.0
somatório 2: 3.0, diagonal: 6.0
True


3.3

In [67]:
print(gauss_seidel(C,[0, 0, 0] ,2))

iteração 1
X_0: [0, 0, 0]
X_1: [0.5        0.9        0.18333334]
Vi_1: [1. 1. 1.]
Var_1: 1.0
iteração 2
X_1: [0.5        0.9        0.18333334]
X_2: [0.90416664 0.90749997 0.04736112]
Vi_2: [0.4470046  0.00826446 2.870967  ]
Var_2: 2.870966911315918
[0.90416664 0.90749997 0.04736112]


7

7.1

In [103]:
G = np.array([[2,-1,0,0,0],[-1,2,-1,0,0],[0,-1,2,-1,0],[0,0,-1,2,128]])
print(G)

[[  2  -1   0   0   0]
 [ -1   2  -1   0   0]
 [  0  -1   2  -1   0]
 [  0   0  -1   2 128]]


7.2

In [116]:
resposta2 = gauss_seidel(G, np.array([0,0,0,0], dtype='f'), 4)
print(f'\nResposta: {resposta2}')

iteração 1
X_0: [0. 0. 0. 0.]
X_1: [ 0.  0.  0. 64.]
Vi_1: [0. 0. 0. 1.]
Var_1: 1.0
iteração 2
X_1: [ 0.  0.  0. 64.]
X_2: [ 0.  0. 32. 80.]
Vi_2: [0.  0.  1.  0.2]
Var_2: 1.0
iteração 3
X_2: [ 0.  0. 32. 80.]
X_3: [ 0. 16. 48. 88.]
Vi_3: [0.         1.         0.33333334 0.09090909]
Var_3: 1.0
iteração 4
X_3: [ 0. 16. 48. 88.]
X_4: [ 8. 28. 58. 93.]
Vi_4: [1.         0.42857143 0.1724138  0.05376344]
Var_4: 1.0

Resposta: [ 8. 28. 58. 93.]


7.3

In [118]:
crit_sassenfeld(G)

b[0] = 0.5
b[1] = 0.75
b[2] = 0.875
b[3] = 0.4375


0.875

8

8.1

In [95]:
H = np.array([[0, 2, 3, 1.5], [4, 2, 0, -2], [2, 2, 0, 0]], dtype='f')
print(H)

[[ 0.   2.   3.   1.5]
 [ 4.   2.   0.  -2. ]
 [ 2.   2.   0.   0. ]]


In [ ]:
gauss_com_pivot(H, 2)

In [70]:
subs_retroativa(H, 2)

iteração 0:
 Matrix antes do passo:
[[ 0.   2.   3.   1.5]
 [ 4.   2.   0.  -2. ]
 [ 2.   2.   0.   0. ]]
Matrix apos arredondamento:
[[ 4.   2.   0.  -2. ]
 [ 0.   2.   3.   1.5]
 [ 0.   1.   0.   1. ]]
iteração 1:
 Matrix antes do passo:
[[ 4.   2.   0.  -2. ]
 [ 0.   2.   3.   1.5]
 [ 0.   1.   0.   1. ]]
Matrix apos arredondamento:
[[ 4.    2.    0.   -2.  ]
 [ 0.    2.    3.    1.5 ]
 [ 0.    0.   -1.5   0.25]]
iteração 2:
 Matrix antes do passo:
[[ 4.    2.    0.   -2.  ]
 [ 0.    2.    3.    1.5 ]
 [ 0.    0.   -1.5   0.25]]
Matrix apos arredondamento:
[[ 4.    2.    0.   -2.  ]
 [ 0.    2.    3.    1.5 ]
 [ 0.    0.   -1.5   0.25]]
saída 3 antes do arredondamento: -0.17000000178813934
saída 3 depois do arredondamento: -0.17000000178813934

saída 2 antes do arredondamento: 1.0049999952316284
saída 2 depois do arredondamento: 1.0

saída 1 antes do arredondamento: -1.0
saída 1 depois do arredondamento: -1.0

saída final:
[-1.    1.   -0.17]


8.2

In [96]:
H[[0,1], :] = H[[1,0],:]
H[[1,2],:] = H[[2,1],:]
print(H)

[[ 4.   2.   0.  -2. ]
 [ 2.   2.   0.   0. ]
 [ 0.   2.   3.   1.5]]


In [97]:
crit_sassenfeld(H)

b[0] = 0.5
b[1] = 0.5
b[2] = 0.3333333432674408


0.5

8.4

In [98]:
resp = gauss_seidel(H, np.array([-1, 1, -.17], dtype='f'), 1)

iteração 1
X_0: [-1.    1.   -0.17]
X_1: [-1.          1.         -0.16666667]
Vi_1: [0.         0.         0.01999998]
Var_1: 0.019999980926513672


In [99]:
erro = (0.5**1/(1-0.5))*(abs(resp[2] - 0.17))
print(erro)

0.33666667163372044
